In [2]:
import json
import pandas as pd
import os

In [118]:
def get_items_and_count(file):
  subcadena = "minecraft:"
  itm_count = {"Items":[],"NItems":[]}
  if "key" in file.keys():
    for patron in file["key"].keys():
      elements = file["key"][patron]
      if type(elements) == list:
        elements = elements[0]
      itm_count["Items"].append(elements[list(elements.keys())[0]].replace(subcadena,""))
      item_count = 0
      for i in range(len(file["pattern"])):
        item_count += file["pattern"][i].count(patron)
      print(item_count)
      itm_count["NItems"].append(item_count)
  elif "ingredients" in file.keys():
    for ingredient in file["ingredients"]:
      if type(ingredient) == list:
        ingredient = ingredient[0]
      itm_count["Items"].append(list(ingredient.values())[0].replace(subcadena,""))
      itm_count["NItems"].append(1)
  elif "ingredient" in file.keys():
    if type(file["ingredient"]) == list:
      ingredient = file["ingredient"][0]
    else:
      ingredient = file["ingredient"]
      itm_count["Items"].append(list(ingredient.values())[0].replace(subcadena,""))
      itm_count["NItems"].append(1)
  return itm_count["Items"],itm_count["NItems"]

In [119]:
def extract_mc_data(file):
  print(file)
  mc_object = {"Object":"","Group":"","Items":[],"NItems":[]}
  if type(file["result"]) == str:
    mc_object["Object"] = file["result"].replace("minecraft:","")
  else:
    mc_object["Object"] = file["result"]["item"].replace("minecraft:","")
  if "group" in file.keys():
    mc_object["Group"] = file["group"]
  else:
    mc_object["Group"] = ""
  mc_object["Items"], mc_object["NItems"] = get_items_and_count(file)
  return mc_object

In [343]:
def read_smithing(file):
  subcadena = "minecraft:"
  itm_count = {"Items":[],"NItems":[],"Proportion":[]}
  itm_count["Items"]=[file["base"]["item"].replace(subcadena,""), file["addition"]["item"].replace(subcadena,"")]
  itm_count["NItems"]=[1,1]
  tot_els = sum(itm_count["NItems"])
  itm_count["Proportion"] = [nit/tot_els for nit in itm_count["NItems"]]
  return itm_count

In [344]:
def read_smelting(file):
  subcadena = "minecraft:"
  itm_count = {"Items":[],"NItems":[],"Proportion":[]}
  if "ingredient" in file.keys():
    if type(file["ingredient"]) == list:
      alt_items = []
      for ingredient in file["ingredient"]:
        alt_items.append(list(ingredient.values())[0].replace(subcadena,""))
      itm_count["Items"].append(alt_items.copy())
      itm_count["NItems"].append(1)
    else:
      ingredient = file["ingredient"]
      itm_count["Items"].append(list(ingredient.values())[0].replace(subcadena,""))
      itm_count["NItems"].append(1)
  tot_els = sum(itm_count["NItems"])
  itm_count["Proportion"] = [nit/tot_els for nit in itm_count["NItems"]]
  return itm_count

In [345]:
def read_crafting_shapeless(file):
  subcadena = "minecraft:"
  itm_count = {"Items":[],"NItems":[],"Proportion":[]}
  for ingredient in file["ingredients"]:
    if type(ingredient) == list:
      alt_items = []
      for i in ingredient:
        alt_items.append(list(i.values())[0].replace(subcadena,""))            
      itm_count["Items"].append(alt_items.copy())
      itm_count["NItems"].append(1)
    else:
      itm_count["Items"].append(list(ingredient.values())[0].replace(subcadena,""))
      itm_count["NItems"].append(1)
  tot_els = sum(itm_count["NItems"])
  itm_count["Proportion"] = [nit/tot_els for nit in itm_count["NItems"]]
  return itm_count

In [346]:
def read_crafting_shape(file):
  subcadena = "minecraft:"
  itm_count = {"Items":[],"NItems":[],"Proportion":[]}
  for patron in file["key"].keys():
    elements = file["key"][patron]
    if type(elements) == list:
      alt_items = []
      for element in elements:
        alt_items.append(element[list(element.keys())[0]].replace(subcadena,""))
      itm_count["Items"].append(alt_items.copy())
      
      item_count = 0
      for i in range(len(file["pattern"])):
        item_count += file["pattern"][i].count(patron)
      itm_count["NItems"].append(item_count)
    else:
      itm_count["Items"].append(elements[list(elements.keys())[0]].replace(subcadena,""))
      item_count = 0
      for i in range(len(file["pattern"])):
        item_count += file["pattern"][i].count(patron)
      itm_count["NItems"].append(item_count)
  tot_els = sum(itm_count["NItems"])
  itm_count["Proportion"] = [nit/tot_els for nit in itm_count["NItems"]]
  return itm_count

In [347]:
def get_items_and_count_refined(file, craft_type):
  if craft_type == "minecraft:smelting":
    itm_count = read_smelting(file)
  elif craft_type == "minecraft:smoking":
    itm_count = read_smelting(file)
  elif craft_type == 'minecraft:stonecutting':
    itm_count = read_smelting(file)
  elif craft_type == 'minecraft:crafting_shaped':
    itm_count = read_crafting_shape(file)
  elif craft_type == 'minecraft:crafting_shapeless':
    itm_count = read_crafting_shapeless(file)
  elif craft_type == 'minecraft:blasting':
    itm_count = read_smelting(file)
  elif craft_type == 'minecraft:campfire_cooking':
    itm_count = read_smelting(file)
  elif craft_type == "minecraft:smithing":
    itm_count = read_smithing(file)
  return itm_count["Items"],itm_count["NItems"],itm_count["Proportion"]

In [348]:
def extract_mc_data(file):
  mc_object = {"Type":"","Object":"","count":1,"Group":"","Items":[],"NItems":[],"Proportion":[]}
  mc_object["Type"] = file["type"]
  if type(file["result"]) == str:
    mc_object["Object"] = file["result"].replace("minecraft:","")
  else:
    mc_object["Object"] = file["result"]["item"].replace("minecraft:","")
    if "count" in file["result"].keys():
      mc_object["count"] = file["result"]["count"]
  if "group" in file.keys():
    mc_object["Group"] = file["group"]
  else:
    mc_object["Group"] = ""
  mc_object["Items"], mc_object["NItems"], mc_object["Proportion"] = get_items_and_count_refined(file, file["type"])
  return mc_object

In [349]:
def extract_mineraft_recipes(directory_path, filter_type):
  archivos = os.listdir(directory_path)
  mc_crafts_data = {"Type":[],"Object":[],"count":[],"Group":[],"Items":[],"NItems":[],"Proportion":[]}
  for archivo in archivos:
      with open(f'{directory_path}/{archivo}') as file:
          data = json.load(file)
          if data["type"] in filter_type:
            mc_object = extract_mc_data(data)
            mc_crafts_data["Type"].append(mc_object["Type"])
            mc_crafts_data["Object"].append(mc_object["Object"])
            mc_crafts_data["count"].append(mc_object["count"])
            mc_crafts_data["Group"].append(mc_object["Group"])
            mc_crafts_data["Items"].append(mc_object["Items"])
            mc_crafts_data["NItems"].append(mc_object["NItems"].copy())
            mc_crafts_data["Proportion"].append(mc_object["Proportion"].copy())
  return mc_crafts_data

In [375]:
def clean_data(mc_crafts_data):
  for i in range(len(mc_crafts_data["Type"])):
    if mc_crafts_data["Type"][i] == "minecraft:crafting_shapeless":
      craft_objcs = mc_crafts_data["Items"][i]
      # print("Objeto suspectible de limpiar")
      # print(mc_crafts_data["Object"][i])
      # print(craft_objcs)
      if all([not isinstance(elemento, list) for elemento in craft_objcs]):
        if len(craft_objcs) != len(set(craft_objcs)):
          clean_item = []
          n_objs = []
          
          for obj in craft_objcs:
            if obj not in clean_item:
              clean_item.append(obj)
              n_objs.append(craft_objcs.count(obj))
          mc_crafts_data["Items"][i] = clean_item
          mc_crafts_data["NItems"][i] = n_objs
          tot_objs = sum(n_objs)
          mc_crafts_data["Proportion"][i] = [nobj/tot_objs for nobj in n_objs]
      # print("Objeto limpio")
      # print(mc_crafts_data["Items"][i])
      # print(mc_crafts_data["NItems"][i])
      # print(mc_crafts_data["Proportion"][i])
      # print("\n")
  return mc_crafts_data

In [394]:
def gen_nodes(mc_crafts_data_df):
  mc_objects_l = list(mc_crafts_data_df["Object"])
  for list_of_items in mc_crafts_data_df["Items"]:
    for item in list_of_items:
      if type(item) == list:
        for ind_item in item:
          if ind_item not in mc_objects_l:
            mc_objects_l.append(ind_item)
      elif item not in mc_objects_l:
        mc_objects_l.append(item)
  group = []
  mc_objects_l = list(set(mc_objects_l))
  for objects in mc_objects_l:
    grouped = False
    for orig_obj in mc_crafts_data_df["Object"]:
      if objects == orig_obj:
        group.append(mc_crafts_data_df[mc_crafts_data_df["Object"] == orig_obj]["Group"].values[0])
        grouped = True
        break
    if not grouped:
      group.append("")
  nodes_data = {"id":list(range(1,len(mc_objects_l)+1)),"Label":mc_objects_l,"group":group}
  nodes_data_pd = pd.DataFrame(nodes_data)
  return nodes_data_pd

In [401]:
def gen_edges(mc_crafts_data_df, nodes_df):
  edges = {"source":[],"target":[],"type":[],"weight":[]}
  for i in range(len(mc_crafts_data_df)):
    label_target = mc_crafts_data_df["Object"][i]
    id_target = nodes_df.loc[nodes_df["Label"] == label_target, "id"].values[0]
    for j in range(len(mc_crafts_data_df["Items"][i])):
      if type(mc_crafts_data_df["Items"][i][j])==list:
        for alt_elm in mc_crafts_data_df["Items"][i][j]:
          label_source = alt_elm
          id_source = nodes_df.loc[nodes_df["Label"] == label_source, "id"].values[0]
          edges["target"].append(id_target)
          edges["source"].append(id_source)
          edges["type"].append("Directed")
          edges["weight"].append(mc_crafts_data_df["Proportion"][i][j])
      else:
        label_source = mc_crafts_data_df["Items"][i][j]
        id_source = nodes_df.loc[nodes_df["Label"] == label_source, "id"].values[0]
        edges["target"].append(id_target)
        edges["source"].append(id_source)
        edges["type"].append("Directed")
        edges["weight"].append(mc_crafts_data_df["Proportion"][i][j])
  return pd.DataFrame(edges)

In [376]:
filter_type= ["minecraft:smelting",
              "minecraft:smoking",
              'minecraft:stonecutting',
              'minecraft:crafting_shaped',
              'minecraft:crafting_shapeless',
              'minecraft:blasting',
              'minecraft:campfire_cooking',
              "minecraft:smithing"]
directorio = 'recipes'
global objects_w_alt
global objects_dw_alt

objects_w_alt = {"Objeto":[], "Alternativas":[]}
objects_dw_alt = {"Objeto":[], "Alternativas":[]}


mc_crafts_data = extract_mineraft_recipes(directorio, filter_type)
mc_crafts_data_df_noclean = pd.DataFrame(mc_crafts_data)
# print(mc_crafts_data_df_noclean)
mc_crafts_data_df_noclean.to_csv("mcdatanoclean.csv")
mc_crafts_data_clean = clean_data(mc_crafts_data)
mc_crafts_data_df = pd.DataFrame(mc_crafts_data_clean)
mc_crafts_data_df.to_csv("extractcrafts_def.csv",index=False)
print(mc_crafts_data_df)

                              Type                     Object  count  \
0        minecraft:crafting_shaped                acacia_boat      1   
1     minecraft:crafting_shapeless              acacia_button      1   
2     minecraft:crafting_shapeless          acacia_chest_boat      1   
3        minecraft:crafting_shaped                acacia_door      3   
4        minecraft:crafting_shaped               acacia_fence      3   
...                            ...                        ...    ...   
1071     minecraft:crafting_shaped       yellow_stained_glass      8   
1072     minecraft:crafting_shaped  yellow_stained_glass_pane     16   
1073     minecraft:crafting_shaped  yellow_stained_glass_pane      8   
1074     minecraft:crafting_shaped          yellow_terracotta      8   
1075  minecraft:crafting_shapeless                yellow_wool      1   

                   Group                     Items  NItems  \
0                   boat           [acacia_planks]     [5]   
1          

In [399]:
nodes_df = gen_nodes(mc_crafts_data_df)
nodes_df.to_csv("nodesdef.csv",index=False)

In [402]:
edges_df = gen_edges(mc_crafts_data_df, nodes_df)
edges_df.to_csv("edgesdef.csv", index = False)